In [1]:
import ee
import geemap as emap
import pandas as pd
import numpy as np

In [2]:
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWgvxw_Sv9H9J6nVHZwMLhPIx6lDq-8UYpHyG-yzJ3w86BfFORP8HrE

Successfully saved authorization token.


In [3]:
Map=emap.Map()

**This notebook aims to extract cloud cover percent over the period of 2015 to 2021.**

- Each image has one band called `quality band` and Landsat 8 named it `QA_PIXEL`. So the idea is to access bits 8-9 and get the high cloud confidence and assigned it to 1. Finally, we can map over all images in the collection and calculate the percentage of cloud confidence per pixel.

## 1. Define Landsat image collection and study map

In [15]:
# Load the study area
vn=ee.FeatureCollection("users/miketu72/VN_Map")
# vn=ee.FeatureCollection("users/miketu72/North_Africa_AOI")
# Landsat 8 collection TOA reflectance 
## Filter all October from 2015 to 2021
ls8=ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA").filterBounds(vn).filter(ee.Filter.calendarRange(2017,2017,"year")).\
filter(ee.Filter.calendarRange(1,1,"month"))

- **Write a function to extract bitmask**

In [16]:
# Write a function to get bitmask
def bitwiseExtract(img, fromBit, toBit):
    maskSize=ee.Number(1).add(toBit).subtract(fromBit)
    mask=ee.Number(1).leftShift(maskSize).subtract(1)
    return img.rightShift(fromBit).bitwiseAnd(mask)

In [17]:
# Write a function to create a binary cloud band =1 (cloud) 0 otherwise
def cloudBand(img):
    fromBit=8
    toBit=9
    qaMask=img.select("QA_PIXEL")
    blue=img.select("B3")
    total=blue.multiply(ee.Number(0)).add(ee.Number(1)).rename("total")
    cloud=bitwiseExtract(qaMask,fromBit,toBit).eq(3).rename("cloud")
    band=img.addBands(cloud).addBands(total)
    return band  
bandMap=ls8.map(cloudBand)

In [18]:
# Select cloud band and sum up
cloudBand=bandMap.select("cloud").sum()
# Select total and sum up
totalBand=bandMap.select("total").sum()
# Calculate the percentage of cloud 
vnCloud=cloudBand.divide(totalBand).multiply(100).clip(vn)

- **Visualizing the cloud percent map**

In [19]:
# Without clipping to Vietnam boundary
cloudPercent=cloudBand.divide(totalBand).multiply(100).clip(vn)
# Define color parameters
Map.setCenter(106.8,16.28,6)
vis_params = {'bands': ['cloud'], 'palette': ['440154', ' 3b528b', ' 21918c', ' 5ec962', ' fde725'], 'min': 0.0, 'max': 100.0}
Map.addLayer(cloudPercent,vis_params,"Landsat Cloud Percent")
Map

Map(bottom=3970.0, center=[16.28, 106.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

## Define Sentinel-2 image collection

In [23]:
# Define the sentinel-2 collection and select all october data in 2015
sentinel2=ee.ImageCollection("COPERNICUS/S2").filterBounds(vn).filter(ee.Filter.calendarRange(2017,2017,"year")).\
filter(ee.Filter.calendarRange(1,1,"month"))

In [24]:
# Write a function to create a binary cloud band =1 (cloud) 0 otherwise
def cloudBand(img):
    fromBit=10
    toBit=10
    qaMask=img.select("QA60")
    blue=img.select("B3")
    total=blue.multiply(ee.Number(0)).add(ee.Number(1)).rename("total") # Create an image with value =1
    cloud=bitwiseExtract(qaMask,fromBit,toBit).eq(1).rename("cloud") # Create a binary image with cloud pixel=1 otherwise 0
    band=img.addBands(cloud).addBands(total) # Add all bands to image
    return band  
sentinelMap=sentinel2.map(cloudBand)

In [25]:
# Select cloud band and sum up
sentinelCloud=sentinelMap.select("cloud").sum()
# Select total band then sum up
sentinelTotal=sentinelMap.select("total").sum()
# Divide cloud pixels over total image taken
sentinelPercent=sentinelCloud.divide(sentinelTotal).multiply(100).clip(vn)

- **Dislay the Sentinel-2 cloud percent map**

In [26]:
vis_params = {'bands': ['cloud'], 'palette': ['440154', ' 3b528b', ' 21918c', ' 5ec962', ' fde725'], 'min': 0.0, 'max': 100.0}
Map.addLayer(sentinelPercent,vis_params,"Sentinel Cloud Percent")
Map

Map(bottom=4086.0, center=[13.496472765758964, 108.14941406250001], controls=(WidgetControl(options=['position…